# PS08 - Is the model fair?
## Name: Kaylee Cho
### Date: 2023/06/01

In [297]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 1. Is compas fair? (50pts)

### 1.1 Load and prepare (2pts)

(1) Laod data and do basic checks.

In [298]:
compas = pd.read_csv("../datasets/compas-score-data.csv.bz2", sep="\t")
compas.head(5)

,age,c_charge_degree,race,age_cat,sex,priors_count,decile_score,two_year_recid
0,69,F,Other,Greater than 45,Male,0,1,0
1,34,F,African-American,25 - 45,Male,0,3,1
2,24,F,African-American,Less than 25,Male,4,4,1
3,44,M,Other,25 - 45,Male,0,1,0
4,41,F,Caucasian,25 - 45,Male,14,6,1


(2)

In [299]:
compas = compas[(compas["race"] == "African-American") | (compas["race"] == "Caucasian")]
compas.head(5)

,age,c_charge_degree,race,age_cat,sex,priors_count,decile_score,two_year_recid
1,34,F,African-American,25 - 45,Male,0,3,1
2,24,F,African-American,Less than 25,Male,4,4,1
4,41,F,Caucasian,25 - 45,Male,14,6,1
6,39,M,Caucasian,25 - 45,Female,0,1,0
7,27,F,Caucasian,25 - 45,Male,0,4,0


(3) Create new dummy variale based off of COMPAS risk score (decile_score), which indicates if an individual was classified as low risk (score 1-4) or high risk (score 5-10).

In [300]:
compas['high_score'] = np.where(compas['decile_score'] <= 4, 0, 1)
compas.head(10)

,age,c_charge_degree,race,age_cat,sex,priors_count,decile_score,two_year_recid,high_score
1,34,F,African-American,25 - 45,Male,0,3,1,0
2,24,F,African-American,Less than 25,Male,4,4,1,0
4,41,F,Caucasian,25 - 45,Male,14,6,1,1
6,39,M,Caucasian,25 - 45,Female,0,1,0,0
7,27,F,Caucasian,25 - 45,Male,0,4,0,0
8,23,M,African-American,Less than 25,Male,3,6,1,1
9,37,M,Caucasian,25 - 45,Female,0,1,0,0
10,41,F,African-American,25 - 45,Male,0,4,0,0
11,47,F,Caucasian,Greater than 45,Female,1,1,1,0
12,31,F,African-American,25 - 45,Male,7,3,1,0


(4) Now analyze the offenders across this new risk category:
(a) What is the recidivism rate (percentage of offenders who re-commit the crime) for low-risk and high-risk individuals?

In [301]:
# (a)
# Calculating low risk recidivism rate
low_risk_re_num = compas[compas['high_score'] == 0].two_year_recid.sum()
low_risk_total = compas[compas['high_score'] == 0].two_year_recid.count()
low_risk_re_rate = (low_risk_re_num/low_risk_total) * 100
# Calulating high risk recidivism rate
high_risk_re_num = compas[compas['high_score'] == 1].two_year_recid.sum()
high_risk_total = compas[compas['high_score'] == 1].two_year_recid.count()
high_risk_re_rate = (high_risk_re_num/high_risk_total) * 100
print("low risk recidivism rate:", low_risk_re_rate,
      "high risk recidivism rate:", high_risk_re_rate)

low risk recidivism rate: 32.00145296040683 high risk recidivism rate: 63.445544554455445


In [302]:
# (b)
# Calculating recidivism rate for African-Americans
afr_re_num = compas[compas['race'] == "African-American"].two_year_recid.sum()
afr_total = compas[compas['race'] == "African-American"].two_year_recid.count()
afr_re_rate = (afr_re_num/afr_total) * 100
# Calculating recidivism rate for Caucasians
cau_re_num = compas[compas['race'] == "Caucasian"].two_year_recid.sum()
cau_total = compas[compas['race'] == "Caucasian"].two_year_recid.count()
cau_re_rate = (cau_re_num/cau_total) * 100
print("African-Americans recidivism rate:", afr_re_rate,
      "Caucasian recidivism rate:", cau_re_rate)

African-Americans recidivism rate: 52.31496062992126 Caucasian recidivism rate: 39.08701854493581


(5) Create CM comparing COMPAS predictions for recidividm (low risk/high risk) and the actual two year recidivism and interpret the results. Recidivists are "positives"! High score is the predicted.

| | **Predicted** | |
| -------- | ------------: | ------:|
| | not recidivists  | recidivists |
|**Actual**| 1872 | 923 |
| | 881 | 1602 |

In [303]:
TN = compas[(compas['high_score'] == 0) & (compas['two_year_recid'] == 0)].high_score.count()
FP = compas[(compas['two_year_recid'] == 0) & (compas['high_score'] == 1)].high_score.count()
FN = compas[(compas['two_year_recid'] == 1) & (compas['high_score'] == 0)].high_score.count()
TP = compas[(compas['high_score'] == 1) & (compas['two_year_recid'] == 1)].high_score.count()
print(TN, FP, FN, TP)

1872 923 881 1602


In [304]:
accuracy = (1872 + 1602)/(TN + FP + FN + TP)
print("accuracy:", accuracy)

accuracy: 0.6582038651004168


In [305]:
FPR = (FP)/(TN + FP)
FPR
FNR = (FN)/(TP + FN)
FNR
print("False positive rate:", FPR, "False negative rate:", FNR)
# FPR - percentage of negatives that are falsely categorized as positives.
# FNR - percentage of positives that are falsely categorized as negatives.

False positive rate: 0.3302325581395349 False negative rate: 0.35481272654047524


According to the result above, roughly 33% of the ppl were categorized as recidivists even though they are not and 35% of ppl were categorized as non recidivists even though they were. I would say researcheres to not use COMPAS beacuse it has high FPR and FNR rate.

#### 1.2 Analysis by race

In [306]:
# (1)
# high risk African American recidivism rate
high_risk_aa = compas[(compas['race'] == "African-American") & (compas['high_score'] == 1)]
high_risk_aa_total = high_risk_aa.two_year_recid.count()
high_risk_aa_re = high_risk_aa[high_risk_aa["two_year_recid"] == 1].two_year_recid.sum()
high_risk_aa_rate = (high_risk_aa_re/high_risk_aa_total) * 100
# low risk African American recidivism rate
low_risk_aa = compas[(compas['race'] == "African-American") & (compas['high_score'] == 0)]
low_risk_aa_total = low_risk_aa.two_year_recid.count()
low_risk_aa_re = low_risk_aa[low_risk_aa["two_year_recid"] == 1].two_year_recid.sum()
low_risk_aa_rate = (low_risk_aa_re/low_risk_aa_total) * 100
print("high risk African American recidivism rate:", high_risk_aa_rate.round(2))
print("low risk African American recidivism rate:", low_risk_aa_rate.round(2))

#high risk Caucasians recidivism rate
high_risk_cc = compas[(compas['race'] == "Caucasian") & (compas['high_score'] == 1)]
high_risk_cc_total = high_risk_cc.two_year_recid.count()
high_risk_cc_re = high_risk_cc[high_risk_cc["two_year_recid"] == 1].two_year_recid.sum()
high_risk_cc_rate = (high_risk_cc_re/high_risk_cc_total) * 100
# low risk Caucasians recidivism rate
low_risk_cc = compas[(compas['race'] == "Caucasian") & (compas['high_score'] == 0)]
low_risk_cc_total = low_risk_cc.two_year_recid.count()
low_risk_cc_re = low_risk_cc[low_risk_cc["two_year_recid"] == 1].two_year_recid.sum()
low_risk_cc_rate = (low_risk_cc_re/low_risk_cc_total) * 100
print("high risk Caucasians recidivism rate:", high_risk_cc_rate.round(2))
print("low risk Caucasians recidivism rate:", low_risk_cc_rate.round(2))

high risk African American recidivism rate: 64.95
low risk African American recidivism rate: 35.14
high risk Caucasians recidivism rate: 59.48
low risk Caucasians recidivism rate: 29.0


(2) Comment the results above. Compare high risk recidivism rate between African Americans and Caucasians. Do the same for low risk group. Is there racial disparity in the result? Which group is favoring? Is Compas fair?

African American tend to have higher high risk and both low risk recidivism rate compared to Caucasians. Regarding this fact since African Americans tend to have higher recidivism rate, we can tell that Compas tends to have more harsh standard towards African American. COMPAS tends to say that African Americans are at higher risk of recidivism than Caucasians according to the above result. Therefore, COMPAS does have racial disparity which shows that COMPAS is not fair.

(3) Repeat your confusion matrix calculation and analysis from 1.1.5 But this time do it separately for African American and for Caucasians.

In [307]:
# confusion matrix for African Americans
compas_aa = compas[compas["race"] == "African-American"].copy()
aa_TN = compas_aa[(compas_aa['high_score'] == 0) & (compas_aa['two_year_recid'] == 0)].high_score.count()
aa_FP = compas_aa[(compas_aa['two_year_recid'] == 0) & (compas_aa['high_score'] == 1)].high_score.count()
aa_FN = compas_aa[(compas_aa['two_year_recid'] == 1) & (compas_aa['high_score'] == 0)].high_score.count()
aa_TP = compas_aa[(compas_aa['high_score'] == 1) & (compas_aa['two_year_recid'] == 1)].high_score.count()
print(aa_TN, aa_FP, aa_FN, aa_TP)

873 641 473 1188


In [308]:
# confusion matrix for Caucasians
compas_cc = compas[compas["race"] == "Caucasian"].copy()
cc_TN = compas_cc[(compas_cc['high_score'] == 0) & (compas_cc['two_year_recid'] == 0)].high_score.count()
cc_FP = compas_cc[(compas_cc['two_year_recid'] == 0) & (compas_cc['high_score'] == 1)].high_score.count()
cc_FN = compas_cc[(compas_cc['two_year_recid'] == 1) & (compas_cc['high_score'] == 0)].high_score.count()
cc_TP = compas_cc[(compas_cc['high_score'] == 1) & (compas_cc['two_year_recid'] == 1)].high_score.count()
print(cc_TN, cc_FP, cc_FN, cc_TP)

999 282 408 414


In [309]:
# Accuracy calculation for both African Americans & Caucasians
accuracy_aa = (aa_TP + aa_TN)/(aa_TN + aa_FP + aa_FN + aa_TP)
accuracy_cc = (cc_TP + cc_TN)/(cc_TN + cc_FP + cc_FN + cc_TP)
print("accuracy for African Americans:", accuracy_aa)
print("accuracy for Caucasians:", accuracy_cc)
# calculating False positive rate (false recividism rate)
FPR_aa = (aa_FP)/(aa_TN + aa_FP)
FPR_cc = (cc_FP)/(cc_TN + cc_FP)
print("false positive rate for African Americans:", FPR_aa)
print("false positive rate for Caucasians:", FPR_cc)
# calculating False no-recidivism rate (false negative rate)
FNR_aa = (aa_FN)/(aa_TP + aa_FN)
FNR_cc = (cc_FN)/(cc_TP + cc_FN)
print("false negative rate for African Americans:", FNR_aa)
print("false negative rate for Caucasians:", FNR_cc)

accuracy for African Americans: 0.6491338582677165
accuracy for Caucasians: 0.6718972895863052
false positive rate for African Americans: 0.4233817701453104
false positive rate for Caucasians: 0.22014051522248243
false negative rate for African Americans: 0.2847682119205298
false negative rate for Caucasians: 0.49635036496350365


(4) As you can see from the results above, false positive rate for African American is almost two times higher than the false positive rate for Caucasians. This means that if you are African American it's two times higher to be called as recidivists even when you are not, compared to Caucasians. Oppositely, according to the false negative rate if you are Caucasian recidivist you have 1.7 times higher chance(50/28) to be called as not recidivists than those who are African American recidivists. Therefore, we can tell that COMPAS is not fair(= biased) especailly in terms of racial bias.

(5) Does your answer in 4 align with your answer in 2? Explain!

Above in question (2) I said that COMPAS tends to have harsh stantards towards African Americans according to the above result. Therefore, I concluded that COMPAS does have racial disparity which shows that COMPAS is not fair. Similarly, in the answer 4, I foudn that COMPAS is racially biased regarding that fact that African Americans tends to have higher FPR and smaller FNR rate compared to Caucasians. As a result, we can tell that COMPAS isn't fair.

### 2 Can you beat COMPAS? (50pts)

#### 2.1 Create the model (30pts)

(1) I would say FPR and FNR rate is a measurement to orive the performance since they tell you the rate of negatives that were falsely categorized as positive and positives that were falsely categorized as negative. Among the two FPR would be more useful since I believe someone being falsely accused as a recidivist would be a bigger problem than the one who is but not accused as one. In my opinion, accusing an innocent seems to be more significant. However, FNR is also important since we don't want to leave recidivists as non recifidivists in the society.

(2) We shouldn't use decile score here since that's the variable column we used to create the high_score which is a predicted data from the first part of the problem set. Since we are trying to create our own prediction with a new model, it is right to exclude that variable that originates from the COMPAS model.

(3)

In [310]:
# (3)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer

# define predictor and response variable
X = compas.drop(['race', 'sex', 'decile_score', 'high_score', 'two_year_recid'], axis=1)
X = pd.get_dummies(X)
y = compas.two_year_recid

#build logistic regression model
m = LogisticRegression(max_iter=100)
_ = m.fit(X, y)
# define cross-validation method to use (cv fits the model for you!)
cv = cross_val_score(m, X, y, cv=10)

# predict the outcome using validation data
val_predict = m.predict(X)
val_predict[:10]
# in the output array 1 denotes recidivist and 0 denotes non-recidivist.

# create confusion matrix
cm = confusion_matrix(y, val_predict)
cm

def my_custom_fpr_function(val_predict:int, y: int) -> float:
    # calculating False positive rate (false recividism rate)
    FPR = np.sum((val_predict == 1) & (y != 1))/(np.sum((val_predict != 1) & (y != 1))+ np.sum((val_predict == 1) & (y != 1)))
    return FPR

def my_custom_fnr_function(val_predict:int, y: int) -> float:
    # calculating False no-recidivism rate (false negative rate)
    FNR = np.sum((val_predict != 1) & (y == 1))/(np.sum((val_predict == 1) & (y == 1)) + np.sum((val_predict != 1) & (y == 1)))
    return FNR

my_scorer_fpr = make_scorer(my_custom_fpr_function, greater_is_better=True)
my_scorer_fnr = make_scorer(my_custom_fnr_function, greater_is_better=True)

# calculating performance score for logistic regression
fpr_score = cross_val_score(m, X, y, scoring=my_scorer_fpr, cv=10).mean()
fnr_score = cross_val_score(m, X, y, scoring=my_scorer_fnr, cv=10).mean()
print("<logistic regression model performance>")
print("false positive rate score:", fpr_score)
print("false negative rate score:", fnr_score)
X

<logistic regression model performance>
false positive rate score: 0.3217154061804506
false negative rate score: 0.3395888346622249


,age,priors_count,c_charge_degree_F,c_charge_degree_M,age_cat_25 - 45,age_cat_Greater than 45,age_cat_Less than 25
1,34,0,1,0,1,0,0
2,24,4,1,0,0,0,1
4,41,14,1,0,1,0,0
6,39,0,0,1,1,0,0
7,27,0,1,0,1,0,0
...,...,...,...,...,...,...,...
6165,30,0,0,1,1,0,0
6166,20,0,1,0,0,0,1
6167,23,0,1,0,0,0,1
6168,23,0,1,0,0,0,1


(4) create different models to find the best model! I'm gonna try trees and k-NN here:)

In [311]:
from sklearn import tree

# model 1: decision trees [classification model]
clf = tree.DecisionTreeClassifier()
_ = clf.fit(X, y)

# define cross-validation method to use (cv fits the model for you!)
cv = cross_val_score(m, X, y, cv=10)

# predict the outcome using validation data
val_predict = m.predict(X)
val_predict[:10]
# in the output array 1 denotes recidivist and 0 denotes non-recidivist.

# create confusion matrix
cm = confusion_matrix(y, val_predict)
cm

def my_custom_fpr_function(val_predict:int, y: int) -> float:
    # calculating False positive rate (false recividism rate)
    FPR = np.sum((val_predict == 1) & (y != 1))/(np.sum((val_predict != 1) & (y != 1))+ np.sum((val_predict == 1) & (y != 1)))
    return FPR

def my_custom_fnr_function(val_predict:int, y: int) -> float:
    # calculating False no-recidivism rate (false negative rate)
    FNR = np.sum((val_predict != 1) & (y == 1))/(np.sum((val_predict == 1) & (y == 1)) + np.sum((val_predict != 1) & (y == 1)))
    return FNR

# calculate performance for model 1
fpr_score = cross_val_score(clf, X, y, scoring=my_scorer_fpr, cv=10).mean()
fnr_score = cross_val_score(clf, X, y, scoring=my_scorer_fnr, cv=10).mean()
print("<decision tree classifier model performance>")
print("false positive rate score:", fpr_score)
print("false negative rate score:", fnr_score)

<decision tree classifier model performance>
false positive rate score: 0.3536954312293652
false negative rate score: 0.35014375338784676


In [312]:
from sklearn.neighbors import KNeighborsRegressor

# model 2: k-nearest neighbors regression
knn = KNeighborsRegressor(n_neighbors=3)
_ = knn.fit(X, y)

# define cross-validation method to use (cv fits the model for you!)
cv = cross_val_score(m, X, y, cv=10)

# predict the outcome using validation data
val_predict = m.predict(X)
print(val_predict[:10])
print(y)
# in the output array 1 denotes recidivist and 0 denotes non-recidivist.

# create confusion matrix
cm = confusion_matrix(y, val_predict)
cm

def my_custom_fpr_function(val_predict:int, y: int) -> float:
    # calculating False positive rate (false recividism rate)
    FPR = np.sum((val_predict == 1) & (y != 1))/(np.sum((val_predict != 1) & (y != 1))+ np.sum((val_predict == 1) & (y != 1)))
    return FPR

def my_custom_fnr_function(val_predict:int, y: int) -> float:
    # calculating False no-recidivism rate (false negative rate)
    FNR = np.sum((val_predict != 1) & (y == 1))/(np.sum((val_predict == 1) & (y == 1)) + np.sum((val_predict != 1) & (y == 1)))
    return FNR

# calculate performance for model 2
fpr_score = cross_val_score(knn, X, y, scoring=my_scorer_fpr, cv=10).mean()
fnr_score = cross_val_score(knn, X, y, scoring=my_scorer_fnr, cv=10).mean()
print("<k-nearest neighbors regression model performance>")
print("false positive rate score:", fpr_score)
print("false negative rate score:", fnr_score)

[0 1 1 0 0 1 0 0 0 1]
1       1
2       1
4       1
6       0
7       0
       ..
6165    1
6166    0
6167    0
6168    0
6170    0
Name: two_year_recid, Length: 5278, dtype: int64
<k-nearest neighbors regression model performance>
false positive rate score: 0.42849700866971074
false negative rate score: 0.319130147643944


false positive rate and false negative rate should be low in a best model which indicates accuracy. According to the results above, we can conclude that logistic regression model has the lowest FPR and FNR score which tells that logistic regression is the best model among the three. logistic regression model is also better than COMPAS model since it has similar FPR rate but lower FNR rate.

#### 2.2 Is your model more fair?

(1) Use your best model to predict the two-year recidivism risk, and compute the percentage of the predicted low-risk and high-risk individuals who recidivate, by race. Is your model more or less fair than COMPAS?

In [313]:
X = compas.drop(['two_year_recid'], axis=1)
X = pd.get_dummies(X)
y = compas.two_year_recid

#build logistic regression model
m = LogisticRegression(max_iter=100)
_ = m.fit(X, y)

# predict the outcome using validation data
val_predict = m.predict(X)
val_predict = pd.Series(val_predict)
val_predict = val_predict.to_frame().rename(columns = {0:'two_year_recid'})

# in the output array 1 denotes recidivist and 0 denotes non-recidivist.
new_compas = pd.concat([X, val_predict], axis=1, join="inner")

# high risk African American recidivism rate
high_risk_aa = new_compas[(new_compas['race_African-American'] == 1) & (new_compas['high_score'] == 1)]
high_risk_aa_total = high_risk_aa.two_year_recid.count()
high_risk_aa_re = high_risk_aa[high_risk_aa["two_year_recid"] == 1].two_year_recid.sum()
high_risk_aa_rate = (high_risk_aa_re/high_risk_aa_total) * 100
# low risk African American recidivism rate
low_risk_aa = new_compas[(new_compas['race_African-American'] == 1) & (new_compas['high_score'] == 0)]
low_risk_aa_total = low_risk_aa.two_year_recid.count()
low_risk_aa_re = low_risk_aa[low_risk_aa["two_year_recid"] == 1].two_year_recid.sum()
low_risk_aa_rate = (low_risk_aa_re/low_risk_aa_total) * 100
print("predicted high risk African American recidivism rate:", high_risk_aa_rate.round(2))
print("predicted low risk African American recidivism rate:", low_risk_aa_rate.round(2))

#high risk Caucasians recidivism rate
high_risk_cc = new_compas[(new_compas['race_Caucasian'] == 1) & (new_compas['high_score'] == 1)]
high_risk_cc_total = high_risk_cc.two_year_recid.count()
high_risk_cc_re = high_risk_cc[high_risk_cc["two_year_recid"] == 1].two_year_recid.sum()
high_risk_cc_rate = (high_risk_cc_re/high_risk_cc_total) * 100
# low risk Caucasians recidivism rate
low_risk_cc = new_compas[(new_compas['race_Caucasian'] == 1) & (new_compas['high_score'] == 0)]
low_risk_cc_total = low_risk_cc.high_score.count()
low_risk_cc_re = low_risk_cc[low_risk_cc["two_year_recid"] == 1].two_year_recid.sum()
low_risk_cc_rate = (low_risk_cc_re/low_risk_cc_total) * 100
print("predicted high risk Caucasians recidivism rate:", high_risk_cc_rate.round(2))
print("predicted low risk Caucasians recidivism rate:", low_risk_cc_rate.round(2))

predicted high risk African American recidivism rate: 43.8
predicted low risk African American recidivism rate: 40.62
predicted high risk Caucasians recidivism rate: 42.81
predicted low risk Caucasians recidivism rate: 39.14


The logistic regression model is more fair than COMPAS model because the gap between the recidivism rate of each race is not large. If you see the COMPAS recidivism rate for each race you can see that African Americans tend to have higher recidivism rate in both high risk and low risk category, but in logistic regression model they are fairly similar.

(2) Compute FPR and FNR by race. Is your model more or less fair than COMPAS?

In [321]:
# data which only includes African-American
compas_aa = compas[compas['race'] == "African-American"].copy()

X_aa = compas_aa.drop(['two_year_recid'], axis=1)
X_aa = pd.get_dummies(X_aa)
y_aa = compas_aa.two_year_recid

#build logistic regression model
m = LogisticRegression(max_iter=100)
_ = m.fit(X_aa, y_aa)

# predict the outcome using validation data
val_predict = m.predict(X_aa)

# create confusion matrix for African American data
cm = confusion_matrix(y_aa, val_predict)
cm

array([[ 988,  526],
       [ 498, 1163]])

In [325]:
# FPR and FNR rate for African American data
aa_TN = 988
aa_FP = 526
aa_FN = 498
aa_TP = 1163
aa_FPR = (aa_FP)/(aa_TN + aa_FP)
aa_FNR = (aa_FN)/(aa_TP + aa_FN)
print("predicted false positive rate who are African American:", aa_FPR)
print("predicted false negative rate who are African American:", aa_FNR)

predicted false positive rate who are African American: 0.3474240422721268
predicted false negative rate who are African American: 0.29981938591210117


In [323]:
# data which only includes Caucasian
compas_cc = compas[compas['race'] == "Caucasian"].copy()

X_cc = compas_cc.drop(['two_year_recid'], axis=1)
X_cc = pd.get_dummies(X_cc)
y_cc = compas_cc.two_year_recid

#build logistic regression model
m = LogisticRegression(max_iter=100)
_ = m.fit(X_cc, y_cc)

# predict the outcome using validation data
val_predict = m.predict(X_cc)

# create confusion matrix for African American data
cm = confusion_matrix(y_cc, val_predict)
cm

array([[1094,  187],
       [ 459,  363]])

In [324]:
# FPR and FNR rate for Caucasian data
cc_TN = 1094
cc_FP = 187
cc_FN = 459
cc_TP = 363
cc_FPR = (cc_FP)/(cc_TN + cc_FP)
cc_FNR = (cc_FN)/(cc_TP + cc_FN)
print("predicted false positive rate who are Caucasian:", cc_FPR)
print("predicted false negative rate who are Caucasian:", cc_FNR)

predicted false positive rate who are Caucasian: 0.14597970335675253
predicted false negative rate who are Caucasian: 0.5583941605839416


Logistic regression model tends to have smaller predicted false positive rate for both races and COMPAS tends to have smaller predicted false negative rate in both race. Therefore, with FPR and FNR rate by each race, I don't think we can precisely tell which model is more fair like we did on the previous question. However, regarding the fact that the gap between predicted false positive rate is larger than the gap between predicted false negative rate I would say logistic regression model is yet a bit more fair than COMPAS model in terms of FPR and FNR rate predicion too.

(3)

With the insights from the above observations, I conclude that logistic regression model, which was selected as the best model among the other two models from 2.1, is more fair than COMPAS knowing that it has better performance in predicting low-risk & high-risk individuals who recidivate and in computing FPR and FNR by race with less bias.

### HOW LONG? It took about 1-2 days full:)